## ACTIVSg grid buildout: Part 1

* works on Kestrel
* conda: buildouts (with pysam=4.1.0 and reV=0.8.2)
* reading in ACTIVSg grid files
* setting up SAM & reV
* running reV
* applying exclusion map
* producing and saving all_cf_df.csv for Part 2

In [1]:
#####
ll2 = [
    "list",
    "to",
    "test",
    "black",
    "formatting",
]

In [2]:
# to see current python kernel in use:
# (too find your kernel in the dropdow do:
##
import sys

sys.executable

'/projects/hpcapps/isatkaus/conda-envs/buildouts/bin/python'

In [3]:
import os
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

import numpy as np
import matplotlib.pyplot as plt
import json


import rex
from rex import Resource

import h5py
import glob

In [4]:
import cufflinks as cl
import pandas as pd
import numpy as np
import time
import os

# not using pywwtk anymore, using eaw wtk data instead
# # set PYWTK_CACHE_DIR to locate WIND Toolkit data
# # will download from AWS as needed
# os.environ["PYWTK_CACHE_DIR"] = os.path.join(os.environ["HOME"], "pywtk-data")

from powerscenarios.parser import Parser
from powerscenarios.grid import Grid

# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", 30)

# plotting (optional)

cl.go_offline()

In [5]:
# voronoi
from shapely.ops import cascaded_union
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
from geovoronoi import voronoi_regions_from_coords, points_to_coords

In [6]:
import geopandas as gpd
import contextily as ctx
import shapely

# same as above but using shapely.Point
from shapely.geometry import Point, LineString, Polygon

from pyproj import Proj, transform
import rasterio
from rasterio.plot import show
import pyproj

* select grid
<a id='select_grid'></a>

[SAM parameters](#sam_parameters)

# load TAMU grid data
* choose TAMU grid - uncomment appropriate line
* grid files can be downloaded from:
* https://electricgrids.engr.tamu.edu/electric-grid-test-cases/

In [7]:
grid_name = "ACTIVSg200"  # TAMU 200 bus case
# grid_name = "ACTIVSg2000"  # TAMU 2000 bus case
# grid_name = 'ACTIVSg10k' # TAMU 10000 bus case
# grid_name = 'ACTIVSg70k' # TAMU 70000 bus case


# path to .aux file (TAMU grids) obtained from e.g.
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/activsg200/
data_dir = "/projects/hpcapps/isatkaus/grid-buildouts-data/grid-data/"
# aux_file_name = data_dir + grid_name + "/" + grid_name + ".aux"
aux_file_name = os.path.join(data_dir, grid_name, grid_name + ".aux")

# parse original .aux file and return dataframes for buses, generators, and wind generators
# here, we need .aux files because those are the only ones with Latitute/Longitude information
parser = Parser()
bus_df, gen_df, wind_gen_df = parser.parse_tamu_aux(aux_file_name)

# see what you got
print("bus_df:")
bus_df
print("gen_df:")
gen_df
print("wind_gen_df:")
wind_gen_df

grid = Grid(grid_name, bus_df, gen_df, wind_gen_df)
grid
print(grid.info())


# branches
# m_filename = data_dir + "/grid-data/" + grid_name + "/case_" + grid_name + ".m"
m_filename = os.path.join(data_dir + grid_name + "/case_" + grid_name + ".m")

# read branch table from scenarios_m_file
table_name = "mpc.branch"
column_names = [
    "fbus",
    "tbus",
    "r",
    "x",
    "b",
    "rateA",
    "rateB",
    "rateC",
    "ratio",
    "angle",
    "status",
    "angmin",
    "angmax",
    "Pf",
    "Qf",
    "Pt",
    "Qt",
    "mu_Sf",
    "mu_St",
    "mu_angmin",
    "mu_angmax",
]

# this table is entirely numeric
numeric_columns = column_names

branch_df = parser.read_m_table(
    m_filename,
    table_name=table_name,
    column_names=column_names,
    numeric_columns=numeric_columns,
)


# rename a few columns to match branch drawing script
branch_df.index.rename("UID", inplace=True)
branch_df.rename(columns={"fbus": "FromBus", "tbus": "ToBus"}, inplace=True)
print("\nbranch_df")
branch_df

# ############usts data
# usts_data_dir = "/Users/isatkaus/projects/ecp/powerscenarios/data/grid-data/USATestSystem/formatted/"

# filename = os.path.join(usts_data_dir, "branch.csv")
# branch_df = pd.read_csv(filename, index_col=0)
# print("\nbranch_df")
# branch_df

bus_df:


,BusNum,BusName,Latitude,Longitude,Zone
0,1,CREVECOEUR0,40.642116,-89.599560,2
1,2,CREVECOEUR1,40.642116,-89.599560,2
2,3,ILLIOPOLIS0,39.866030,-89.251291,4
3,4,ILLIOPOLIS1,39.866030,-89.251291,4
...,...,...,...,...,...
196,197,GIBSONCITY12,40.470500,-88.398700,6
197,198,MAPLETON0,40.556403,-89.762073,2
198,199,GIFFORD0,40.306027,-87.958606,6
199,200,PETERSBURG0,40.169931,-89.986537,4


gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,49,1,4.530000,1.360000,1.0,Coal,49_Coal_1,RANTOUL21,40.312222,-88.159444,6
1,50,1,4.530000,1.360000,1.0,Coal,50_Coal_1,RANTOUL22,40.312222,-88.159444,6
2,51,1,4.530000,1.360000,1.0,Coal,51_Coal_1,RANTOUL23,40.312222,-88.159444,6
3,52,1,4.530000,1.360000,1.0,Coal,52_Coal_1,RANTOUL24,40.312222,-88.159444,6
...,...,...,...,...,...,...,...,...,...,...,...
45,183,1,26.600000,7.980000,1.0,Coal,183_Coal_1,SPRINGFIELD22,39.806868,-89.641653,4
46,189,1,569.150019,170.749998,1.0,Nuclear,189_Nuclear_1,CLINTON12,40.171900,-88.833900,7
47,196,1,67.500001,20.250000,1.0,NaturalGas,196_NaturalGas_1,GIBSONCITY11,40.470500,-88.398700,6
48,197,1,67.500001,20.250000,1.0,NaturalGas,197_NaturalGas_1,GIBSONCITY12,40.470500,-88.398700,6


wind_gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,65,1,150.399995,45.120001,1.0,Wind,65_Wind_1,PAXTON11,40.46405,-88.021517,6
1,104,1,99.000001,29.699999,1.0,Wind,104_Wind_1,ELLSWORTH12,40.47920,-88.798900,7
2,105,1,198.000002,59.400004,1.0,Wind,105_Wind_1,ELLSWORTH13,40.47920,-88.798900,7
3,114,1,1.700000,0.510000,1.0,Wind,114_Wind_1,NORMAL22,40.53700,-89.019000,7
4,115,1,150.000000,44.999999,1.0,Wind,115_Wind_1,NORMAL23,40.53700,-89.019000,7
5,147,1,100.500000,30.149999,1.0,Wind,147_Wind_1,HOPEDALE21,40.36920,-89.402200,7


Grid(name=ACTIVSg200, buses=200, generators=49, wind_generators=6, wind_sites=0)


ACTIVSg200 grid info: 

 number of buses: 200
 number of generators: 49
 number of wind generators: 6
 number of solar generators: 0
 total generator capacity: 3602.84 MW
 wind capacity/penetration: 699.60 MW / 19.42%
 solar capacity/penetration: 0.00 MW / 0.00%

branch_df


,FromBus,ToBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax
UID,,,,,,,,,,,,,,,,,,,,,
0,2,1,0.000673,0.003339,0.00000,100.0,0,0,0,0,1,0,0,-7.39,-2.10,7.39,2.11,0.0,0.0,0.0,0.0
1,1,119,0.018542,0.119758,0.02285,221.1,0,0,0,0,1,0,0,12.25,-0.57,-12.22,-1.63,0.0,0.0,0.0,0.0
2,124,1,0.005615,0.036268,0.00692,221.1,0,0,0,0,1,0,0,40.13,7.94,-40.04,-8.08,0.0,0.0,0.0,0.0
3,193,1,0.004258,0.027501,0.00525,221.1,0,0,0,0,1,0,0,-20.39,-6.96,20.41,6.54,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,188,187,0.001394,0.044762,0.00000,400.0,0,0,1,0,1,0,0,-113.10,8.78,113.27,-3.48,0.0,0.0,0.0,0.0
242,189,187,0.000300,0.007815,0.00000,740.0,0,0,1,0,1,0,0,384.37,-24.15,-383.96,34.87,0.0,0.0,0.0,0.0
243,196,195,0.002850,0.060038,0.00000,87.7,0,0,1,0,1,0,0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
244,197,195,0.002850,0.191484,0.00000,87.7,0,0,1,0,1,0,0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0


In [8]:
# distance between lat,lon points
import math


def distance(lat1, lon1, lat2, lon2):
    lat1 = math.radians(abs(lat1))
    lon1 = math.radians(abs(lon1))
    lat2 = math.radians(abs(lat2))
    lon2 = math.radians(abs(lon2))

    R = 6373.0
    # radius of the Earth
    # change in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


# calculate line distances
# add to/from lat/lon to the branch df
# def distance(lat1, lon1, lat2, lon2):


for line_uid in branch_df.index:
    from_latitude = bus_df.set_index("BusNum").loc[
        branch_df.loc[line_uid, "FromBus"], "Latitude"
    ]
    from_longitude = bus_df.set_index("BusNum").loc[
        branch_df.loc[line_uid, "FromBus"], "Longitude"
    ]
    to_latitude = bus_df.set_index("BusNum").loc[
        branch_df.loc[line_uid, "ToBus"], "Latitude"
    ]
    to_longitude = bus_df.set_index("BusNum").loc[
        branch_df.loc[line_uid, "ToBus"], "Longitude"
    ]

    branch_df.loc[line_uid, "distance"] = distance(
        from_latitude, from_longitude, to_latitude, to_longitude
    )

    branch_df.loc[line_uid, "from_latitude"] = from_latitude
    branch_df.loc[line_uid, "from_longitude"] = from_longitude
    branch_df.loc[line_uid, "to_latitude"] = to_latitude
    branch_df.loc[line_uid, "to_longitude"] = to_longitude

##  gdfs for plotting

In [ ]:
#####################################
# bus_df
####################################

b_df = bus_df[["BusNum", "Latitude", "Longitude"]].set_index("BusNum").copy()
# “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
bus_gdf = gpd.GeoDataFrame(
    b_df,
    geometry=gpd.points_from_xy(b_df["Longitude"], b_df["Latitude"]),
    crs="epsg:4326",
)
bus_gdf


# OPTIONAL: make bus "sizes" reflect the number of buses at the same latlon
df = bus_gdf.copy()
df["subgraph"] = 1


df = (
    df[["Latitude", "Longitude", "subgraph"]]
    .groupby(["Latitude", "Longitude"])
    .transform("count")
)
df

bus_gdf["size"] = df["subgraph"] * 10
bus_gdf

# unique latlons
bus_unique_gdf = bus_gdf.drop_duplicates(["Latitude", "Longitude"])
print("\nnumber of unique latlons: {} out of 2000".format(len(bus_unique_gdf)))
bus_unique_gdf


####

###########################################
# branch_df
##############################################

br_df = branch_df[
    [
        "FromBus",
        "ToBus",
        "from_latitude",
        "from_longitude",
        "to_latitude",
        "to_longitude",
    ]
].copy()

# make from/to Points

br_df["FromPoint"] = [
    Point(x, y) for x, y in zip(br_df["from_longitude"], br_df["from_latitude"])
]
br_df["ToPoint"] = [
    Point(x, y) for x, y in zip(br_df["to_longitude"], br_df["to_latitude"])
]


# lines
br_df["geometry"] = [
    LineString([x1, x2]) for x1, x2 in zip(br_df["FromPoint"], br_df["ToPoint"])
]


# “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
br_gdf = gpd.GeoDataFrame(br_df, geometry="geometry", crs="epsg:4326")

br_gdf.plot()

br_df


bus_gdf = bus_gdf.to_crs(epsg=3857)
br_gdf = br_gdf.to_crs(epsg=3857)

# experiment dir
* select run_dir for reV runs: input and output
* change msg describing experiment for logging purposes


In [ ]:
# run_dir = "/projects/exasgd/isatkaus/buildout/wind/wind_gen_tamu5"
# run_dir = "/home/isatkaus/projects/hpcapps/grid-buildouts-data/rev-runs/wind-gen-tamu200-v5"
# run_dir = "/home/isatkaus/projects/hpcapps/grid-buildouts-data/rev-runs/wind-gen-tamu200-v6"
# run_dir = "/home/isatkaus/projects/hpcapps/grid-buildouts-data/rev-runs/wind-gen-tamu200-v7"
import datetime
import reV
import PySAM

run_dir = (
    "/home/isatkaus/projects/hpcapps/grid-buildouts-data/rev-runs/wind-gen-tamu200-v8"
)

# ### test
# run_dir = "/home/isatkaus/projects/hpcapps/grid-buildouts-data/rev-runs/wind-gen-tamu200-test"

if not os.path.exists(run_dir):
    os.makedirs(run_dir)


now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

msg = f"""
TESTING
experiment date: {now}
tamu200 grid, all years 
two ATB-mid30 turbines per WTK cell 
nrel-pysam version {PySAM.__version__}
nrel-reV verion: {reV.__version__}
python version: {sys.version}
using 5-min resource files (all files with * for the height)
"""

# quick readme to know what experiment we're running
filename = os.path.join(run_dir, "readme.txt")
with open(filename, "w") as f:
    f.write(msg)

print(f"\n writting msg, to {filename}: \n")
print(msg)

# project points
* select years of data that will be used in the reV run below. 
* it is recommended to use all available, e.g., WTK has 7 years 2007-2013
* also select resourse file res_file:
* here we just need one file (one year, one hh) to extract gid's of sites
* that we will use to run reV on (project points)



In [ ]:
run_dir
0.424 * 140

In [ ]:
# # on HSDS
# res_file = "/nrel/wtk/conus-5min/wtk_conus_2007.h5"
# res = rex.resource_extraction.resource_extraction.ResourceX(res_file, hsds=True)

# wind resource file on Eagle
# res_file = "/lustre/eaglefs/OLD-datasets/WIND/conus/v1.0.0/wtk_conus_2007.h5"
year = 2007
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013]
# years = [2007,2008]

# this is on lustre file sytem (works)
# res_file = f"/lustre/eaglefs/OLD-datasets/WIND/conus/v1.0.0/wtk_conus_{year}.h5"


# ### vast flash-based system (works) HOURLY
# res_file = f"/vast/datasets/WIND/conus/v1.0.0/wtk_conus_{years[0]}.h5"

# ### vast flash-based system (works) 5-MIN
# res_file = f"/vast/datasets/WIND/conus/v1.0.0/{years[0]}/wtk_conus_{years[0]}_120m.h5"

# Kestrel 5-min, lustre (there is no vast flash-based on Kestrel)
res_file = f"/kfs2/pdatasets/WIND/conus/v1.0.0/{years[0]}/wtk_conus_{years[0]}_120m.h5"


res = rex.resource_extraction.resource_extraction.ResourceX(
    res_file,
)


# # small box
# #gids = res.box_gids((37, -120), (36.7, -119.7))
# gids = res.box_gids((34.55, -117.2), (34.24, -116.86)) # 259 points

# # RTS bounding box
# gids = res.box_gids((37, -120), (32.5, -112))


# create TAMU box, rectangle that includes entire grid selected above
extra_lat = 0.3
extra_lon = 0.3
top = bus_df["Latitude"].max() + extra_lat
bottom = bus_df["Latitude"].min() - extra_lat
right = bus_df["Longitude"].max() + extra_lon
left = bus_df["Longitude"].min() - extra_lon

gids = res.box_gids((top, left), (bottom, right))


pp_df = pd.DataFrame(index=gids)
pp_df.index.name = "gid"
# 'default' will be defined in config_gen.json
pp_df["config"] = "default"
pp_df


# #scratch dir
# df.to_csv('/scratch/isatkaus/projects/dispatches/experiments/full_pipeline_execution/pp.csv')
# df.to_csv('/scratch/isatkaus/projects/dispatches/experiments/wind_gen/pp.csv')
pp_df.to_csv(os.path.join(run_dir, "pp.csv"))

In [ ]:
run_dir

In [ ]:
# save project points with lat/lons for possible plotting
# get mata data of the resource (we need latlons)
meta_df = res.meta
meta_df
filename = "meta_df.csv"
meta_df.to_csv(os.path.join(run_dir, filename))
# get latlons of pp points only for visualization
pps_df = meta_df.loc[pp_df.index, ["latitude", "longitude"]]
pps_df

# save pps_df
run_dir
filename = "pps_df.csv"
pps_df.to_csv(os.path.join(run_dir, filename))

[Select grid](#select_grid)

* sam parameters
<a id='sam_parameters'></a>

[Run ReV](#run_rev)

# SAM file: select tecnology
* create SAM json describing technology inside WTK grid cell
* can use SAM GUI to create .json file 
* or write/modify manually (e.g. ATB turbines are not in SAM library) 
* we'll use ATB 2030 land based 5.5 MW wind example input just add second turbine
* technically, adding a scond turbine is not necessary, can use one, but then multiply the output by capacity density (as we do with solar)
* ATB (Annual Technology Baseline)


In [ ]:
# # standard SAM input used by reV
# sam_filename_in = "/home/isatkaus/projects/dispatches/reV/tests/data/SAM/wind_gen_standard_losses_0.json"
# sam_filename_out = os.path.join(run_dir,"wind_gen1.json")

# SAM file (ATB-mid turbine)
sam_filename_in = "/projects/hpcapps/isatkaus/grid-buildouts-data/sam-templates/land_3_2030_midcost_medinnov.json"
sam_filename_out = os.path.join(run_dir, "wind_gen_2atb_turbine.json")

with open(sam_filename_in) as f:
    data = json.load(f)

data

# add second turbine
data["wind_farm_xCoordinates"].append(1000)
data["wind_farm_yCoordinates"].append(1000)
data["system_capacity"] = data["system_capacity"] * 2
data
with open(sam_filename_out, "w") as f:
    json.dump(data, f, indent=2, sort_keys=True)

# reV generation config
* setup reV generation module run: number of nodes, allocation, resourse files (5-min, 1-hr, hub_height, etc), output data request..
* use lustre file system for large runs, else vast is fine
* make sure to change to your HPC allocation (execution_control)
* see docs at:
* https://nrel.github.io/reV/_cli/reV%20generation.html

In [ ]:
import reV

reV.__version__
run_dir

In [ ]:
#################################
# these are for nrel-rev=0.8.2
#################################

config_filename_in = (
    "/projects/hpcapps/isatkaus/grid-buildouts-data/rev-configs/config_gen.json"
)
config_filename_out = os.path.join(run_dir, "config_gen.json")

with open(config_filename_in) as f:
    data = json.load(f)

data

# "directories" keyword arg is depreciated (works in older reV, only)
data.pop("directories", None)
data["log_directory"] = "./logs/"

# data["analysis_years"] = [years[0]]
data["analysis_years"] = years

# execution: pick YOUR allocation
# data["execution_control"]["allocation"] = "hpcapps"
data["execution_control"]["allocation"] = "exasgd"
# data["execution_control"]["allocation"] = "aiuserapps"
#
# data['execution_control']['feature']='--qos=high' # this key is in the default
# data['execution_control']['feature']='--qos=normal'
data["execution_control"].pop("feature", None)

data["execution_control"]["qos"] = "normal"
# data["execution_control"]["qos"] = "standby"
data["execution_control"]["walltime"] = 4.0
data["execution_control"]["nodes"] = 20

# pick compute sytem
data["execution_control"]["option"] = "kestrel"
# data["execution_control"]["option"] = "eagle"
# data["execution_control"]["option"] = "slurm"
# data["execution_control"]["option"] = "local"

# SAM input
data["sam_files"]["default"] = sam_filename_out

# SAM output
data["output_request"] = [
    "cf_mean",
    "cf_profile",
    "gen_profile",
    "lcoe_fcr",
    "ws_mean",
    "annual_energy",
]


# project points
data["project_points"] = "pp.csv"

# resource
# res_file = f"/vast/datasets/WIND/conus/v1.0.0/wtk_conus_{years[0]}.h5"
# res_file1 = f"/vast/datasets/WIND/conus/v1.0.0/wtk_conus_{years[1]}.h5"

# lustre datasets are now in OLD-datasets:
# /lustre/eaglefs/OLD-datasets/
# data["resource_file"] = (
#     "/lustre/eaglefs/datasets/WIND/conus/v1.0.0/"
#     + str(year)
#     + "/wtk_conus_"
#     + str(year)
#     + "_*.h5"
# )

# vast
# #### these are HOURLY
# data["resource_file"]=[f"/vast/datasets/WIND/conus/v1.0.0/wtk_conus_{year}.h5" for year in years]

# #### these are 5-MIN
# data["resource_file"]=[f"/vast/datasets/WIND/conus/v1.0.0/{year}/wtk_conus_{year}_120m.h5" for year in years]

# #### these are 5-MIN (all heights)
# data["resource_file"] = [
#     f"/vast/datasets/WIND/conus/v1.0.0/{year}/wtk_conus_{year}_*.h5" for year in years
# ]

# on Kestrel (all lustre)
# these are 5-MIN (all heights, all years)
data["resource_file"] = [
    f"/kfs2/pdatasets/WIND/conus/v1.0.0/{year}/wtk_conus_{year}_*.h5" for year in years
]

# ############################# on Kestrel one year (all lustre)
# ####  these are 5-MIN (all heights, one year)
# data["resource_file"] = [
#     f"/kfs2/pdatasets/WIND/conus/v1.0.0/{years[0]}/wtk_conus_{years[0]}_*.h5"]


data["technology"] = "windpower"

data


with open(config_filename_out, "w") as f:
    json.dump(data, f, indent=2, sort_keys=True)

[SAM parameters](#sam_parameters)

* run reV commands
<a id='run_rev'></a>


[Apply exclusions](#apply_exclusions)

# on Kestrel's command line:
* will run all "years"

ssh to kestrel via terminal

cd to experiment dir: run_dir

module load anaconda3 (default 2022)
* or

module load anaconda3/2023.07-2

conda activate buildouts

reV generation -c config_gen.json (nrel-rev=0.8.2) 

* monitor your batch jobs, proceed only after they finished:

squeue -u $USER

In [ ]:
run_dir

# processing .h5
* after batch jobs complete, run_dir will contain .h5 files with data
* based on config_gen.json the output will have N (e.g. N=20 nodes) .h5 files, each containing ~equal number of gid's

In [ ]:
run_dir

In [ ]:
!cat {run_dir}/readme.txt

In [ ]:
!ls {run_dir}

### one file
* sanity check: take a look at one output file


In [ ]:
# can base the buildout (cap factors) on whichever year (out of 2007-2013)
year = 2007

# for reV=0.8.2 naming convention
filename = os.path.join(
    run_dir, run_dir.split("/")[-1] + "_generation_j000_" + str(year) + ".h5"
)

# display all available keys in the data
f = h5py.File(filename, "r")
print("\navailable data keys")
f.keys()

# display a few datasets by chosing a key
key = "meta"
print(f"\n {key}")
dataset = f.get(key)
df = pd.DataFrame(dataset[()])
df

key = "time_index"
print(f"\n {key}")
dataset = f.get(key)
df = pd.DataFrame(dataset[()])
df

key = "gen_profile"
print(f"\n {key}")
dataset = f.get(key)
df = pd.DataFrame(dataset[()])
df

#  read in .h5 and aggregate to one df: scalar field

* since generated data (each year) is split into N = `number of nodes in reV config` .h5 files (N=20), want to aggregate all gids to into one df
* e.g. cf_mean
* based on one year

In [ ]:
# scalar output like cf_mean,
# base_dir = "/home/isatkaus/projects/dispatches/experiments/full_pipeline_execution/"
# glob.glob("/home/isatkaus/projects/dispatches/experiments/full_pipeline_execution/*.h5")

# scratch dir
# base_dir = "/scratch/isatkaus/projects/dispatches/experiments/full_pipeline_execution/"
# base_dir = "/scratch/isatkaus/projects/dispatches/experiments/wind_gen/"

year = 2007

base_dir = run_dir

field_name = "cf_mean"
all_df = pd.DataFrame()
# for rev=0.8.2
for k, filename in enumerate(
    glob.glob(os.path.join(base_dir, "*_generation_*{}.h5".format(year)))
):
    #     if k==1:
    #         break
    # #     k
    # #     filename
    with h5py.File(filename, "r") as f:
        meta = f.get("meta")
        meta_df = pd.DataFrame(meta[()])
        index = meta_df["gid"]
        latitude = meta_df["latitude"]
        longitude = meta_df["longitude"]

        #         time_index = f.get('time_index')
        #         time_df = pd.DataFrame(time_index[()])
        #         time_index = pd.to_datetime(time_df[0].map(lambda x : str(x)[2:-1]))

        df = pd.DataFrame()
        dataset = f.get(field_name)
        df[field_name] = dataset[()]

        # add lat,lon info
        df["latitude"] = meta_df["latitude"]
        df["longitude"] = meta_df["longitude"]

        # add annual energy, covert to MWh
        dataset = f.get("annual_energy")
        df["generation_mwh"] = dataset[()] / 1000

        #         #
        #         dataset = f.get('gen_profile')
        #         df = pd.DataFrame(dataset[()])
        #         df
        #         df.sum()

        df.index = index
        all_df = pd.concat([all_df, df], axis=0)

cf_df = all_df
cf_df

# (optional) read the actual input to reV file: resource file

* uncomment for sanity checks, debugging, curiousity, etc.
* wtk_conus_2007.h5 files are hourly
* for five minute timeseries use
* /kfs2/pdatasets/WIND/conus/v1.0.0/2008/

In [ ]:
# filename = "/kfs2/pdatasets/WIND/conus/v1.0.0/wtk_conus_2007.h5"


# f = h5py.File(filename, "r")
# print("\navailable data")
# f.keys()

# # dataset = f.get('lcoe_fcr')
# # df = pd.DataFrame(dataset[()])
# # df
# key = "meta"
# print(f"\n {key}")
# dataset = f.get(key)
# df = pd.DataFrame(dataset[()])
# df

# key = "time_index"
# print(f"\n {key}")
# dataset = f.get(key)
# df = pd.DataFrame(dataset[()])
# df

In [ ]:
###############
# for gdp viz

# make gdf with project points
cf_df["geometry"] = [Point(x, y) for x, y in zip(cf_df["longitude"], cf_df["latitude"])]
# cf_df
# “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
cf_gdf = gpd.GeoDataFrame(cf_df, geometry="geometry", crs="epsg:4326")
cf_gdf = cf_gdf.to_crs("epsg:3857")

## cf cutoff
* capacity factor histogram based on one year 

In [ ]:
cf_df["cf_mean"].iplot(kind="hist", title="histogram of capacity factors", bins=100)

## plot capacity factors
* sanity check

In [ ]:
# # make gdf with project points
# cf_df["geometry"] = [Point(x, y) for x, y in zip(cf_df["longitude"], cf_df["latitude"])]
# # cf_df
# # “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
# cf_gdf = gpd.GeoDataFrame(cf_df, geometry="geometry", crs="epsg:4326")

cf_gdf = cf_gdf.to_crs("epsg:3857")
# plt.figure(figsize=(20, 20))
ax = cf_gdf.plot(
    figsize=(20, 20),
    markersize=70,
    column="cf_mean",
    legend=True,
    legend_kwds={
        "shrink": 0.6,
    },
    alpha=0.5,
    cmap="viridis",
)


# Web Mercator
# rts_box_gdf = rts_box_gdf.to_crs(epsg=3857)
bus_gdf = bus_gdf.to_crs(epsg=3857)
br_gdf = br_gdf.to_crs(epsg=3857)
# bus_gdf.plot(ax=ax, color="black",  markersize = 5)
bus_gdf.plot(ax=ax, color="black", markersize="size")

br_gdf.plot(ax=ax, color="black", linewidth=1)


# based on project points, draw a box (OPTIONAL)
top = pps_df["latitude"].max()
bottom = pps_df["latitude"].min()
left = pps_df["longitude"].min()
right = pps_df["longitude"].max()

lat_point_list = [top, top, bottom, bottom]
lon_point_list = [left, right, right, left]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
# “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
tamu_box_gdf = gpd.GeoDataFrame(index=[0], crs="epsg:4326", geometry=[polygon_geom])
# print(polygon.geometry)

tamu_box_gdf = tamu_box_gdf.to_crs("epsg:3857")
tamu_box_gdf.boundary.plot(ax=ax, alpha=0.5, edgecolor="k")


# ####### add map tile stamen tile service is retiring
# ctx.add_basemap(ax)

# By default, contextily uses the Stamen Terrain style. We can specify a different style using "source" kwarg
# full list in ctx.sources module or
# https://github.com/geopandas/contextily


ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
# ctx.add_basemap(ax,source=ctx.providers.OpenTopoMap)
# ctx.add_basemap(ax,source=ctx.providers.Stamen.Terrain)
# ctx.add_basemap(ax,source=ctx.providers.Stamen.Toner)


# ### save figure in work dir (since this is the same for all buildout options)
# ax.set_axis_off()
# filename = os.path.join(work_dir, "cfs_all.png")
# plt.savefig(filename, bbox_inches="tight", pad_inches=0)

## high df
* can skipp, will do this in Part2

In [ ]:
# # select all sites in the box, selection by distance will be done later
# selected_df = cf_df.copy()

# # # capacity factor cutoff (old reV)
# # cf_cutoff = 400
# # capacity factor cutoff (new reV)
# cf_cutoff = 0.40

# high_df = selected_df.loc[selected_df["cf_mean"] >= cf_cutoff]
# low_df = selected_df.loc[selected_df["cf_mean"] < cf_cutoff]

# # color_map = ["blue" if x < cf_cutoff else "red" for x in selected_df["cf_mean"]]
# # len(color_map)


# # print('RTS total load (MWh):{:,}'.format(36581383.0))
# total_generation = high_df["generation_mwh"].sum()
# print(
#     "total generation of selected high capacity sites (MWh):{:,}".format(
#         round(total_generation)
#     )
# )

# # total_generation /36581383.0
# print("\nhigh_df:")
# high_df
# print("\nlow_df:")
# low_df

# ###############
# #### for gdp viz

# # make gdf with project points
# high_df["geometry"] = [Point(x, y) for x, y in zip(high_df["longitude"], high_df["latitude"])]
# # cf_df
# # “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
# high_gdf = gpd.GeoDataFrame(high_df, geometry="geometry", crs="epsg:4326")
# high_gdf = high_gdf.to_crs("epsg:3857")

### save/load checkpoint (optional)
* uncomment next two cells as needed

In [ ]:
# ### save high_df and low_df
# run_dir
# filename = "high_cf_df.csv"
# high_df.to_csv(os.path.join(run_dir, filename))
# filename = "low_cf_df.csv"
# low_df.to_csv(os.path.join(run_dir, filename))

In [ ]:
# ### load
# filename = "high_cf_df.csv"
# high_df = pd.read_csv(os.path.join(run_dir, filename), index_col=0)


# # make gdf with project points
# high_df["geometry"] = [
#     Point(x, y) for x, y in zip(high_df["longitude"], high_df["latitude"])
# ]
# # cf_df
# # “EPSG:4326” WGS84 Latitude/Longitude, used in GPS
# high_gdf = gpd.GeoDataFrame(high_df, geometry="geometry", crs="epsg:4326")
# high_gdf = high_gdf.to_crs("epsg:3857")

# high_gdf

[Run reV](#run_rev)

* apply exclusions
<a id='apply_exclusions'></a>

[Go to the end](#the_end)

# inclusion/exclusion

 * select your exclusion mask
 * pip install rasterio to read .tif files
 * projection used: North America Albers Equal Area Conic
 https://spatialreference.org/ref/esri/102008/
 * pip install pyproj to transform (lat,lon) to .tif file coordinates
 * in tif files: 0 is fully excluded, 100 fully included

### run once, takes a while
* might cause memory error
* if causes MemoryError (Unable to allocate 6.12 GiB for an array with shape (33792, 48640) and data type float32), comment out: show(img), plt.figure(), and plt.imshow()
* 

In [ ]:
# mask file
fp = "/projects/hpcapps/isatkaus/grid-buildouts-data/exclusion-masks/05_b_b_mid_inclusion_mask_inclusion_mask.tif"

img = rasterio.open(fp)
# show(img)

dataset = rasterio.open(fp)

# # read into np n-d array
band1 = dataset.read(1)
# plt.figure(figsize=(5, 10))
plt.imshow(band1)

In [ ]:
# p = pyproj.Proj(
#     "+proj=merc +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=6378137 +b=6378137 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
# )

# ESRI North America Albers equal area conic projection
p = pyproj.Proj(
    "+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs"
)

## one cell
* take a look at one gid cell with exclusions applied 

In [ ]:
# lat on of one WTK gid=413850
# lat0, lon0 = 33.686546, -114.395737

# cf_idx = 379449
cf_idx = cf_df.index[2006]
cf_idx = cf_df.index[2007]
cf_idx = cf_df.index[2016]

lat0, lon0 = tuple(cf_df.loc[cf_idx, ["latitude", "longitude"]].values)

# to Albers
x0, y0 = p(lon0, lat0, inverse=False)
# x0, y0 are coordinates of the center of WTK cell

# x y  left top corner
x1 = x0 - 1000
y1 = y0 + 1000

# x y  right bottom corner
x2 = x0 + 1000
y2 = y0 - 1000


# to index
row1, col1 = dataset.index(x1, y1)
row2, col2 = dataset.index(x2, y2)

row1, col1
row2, col2

cell = band1[row1:row2, col1:col2]
plt.figure(figsize=(10, 10))
plt.imshow(cell)

# #### save fig
# save_dir = os.path.join(run_dir,"figures")
# filename = "one_cell_with_exclusions.png"
# plt.savefig(os.path.join(save_dir,filename),  bbox_inches='tight',   )

In [ ]:
actual_inclusion = cell.sum()
actual_inclusion
possible_inclusion = cell.shape[0] * cell.shape[1] * 100
possible_inclusion
print("inclusion fraction: {}".format(round(actual_inclusion / possible_inclusion, 2)))

# all cells: inclusion/exclusion
* go cell by cell, record inclusion
* will take a while
* will use inclusion column in part2
* then save all_cf_df.csv, and proceed to part2

### apply exclusions to all cf

In [ ]:
cf_gdf

In [ ]:
for cf_idx in cf_gdf.index[:]:
    lat0, lon0 = tuple(cf_gdf.loc[cf_idx, ["latitude", "longitude"]].values)

    # to Albers
    x0, y0 = p(lon0, lat0, inverse=False)
    # x0, y0 are coordinates of the center of WTK cell

    # x y  left top corner
    x1 = x0 - 1000
    y1 = y0 + 1000

    # x y  right bottom corner
    x2 = x0 + 1000
    y2 = y0 - 1000

    # to index
    row1, col1 = dataset.index(x1, y1)
    row2, col2 = dataset.index(x2, y2)

    #     row1, col1
    #     row2, col2

    cell = band1[row1:row2, col1:col2]

    total_points = cell.shape[0] * cell.shape[1] * 100
    incl_percent = cell.sum() / (cell.shape[0] * cell.shape[1] * 100)
    # incl_percent
    cf_gdf.loc[cf_idx, "inclusion"] = incl_percent

cf_gdf

In [ ]:
# save all_cf_df with inclusion percentages
run_dir
filename = "all_cf_df.csv"
cf_gdf.to_csv(os.path.join(run_dir, filename))

# Proceed to Part2
* can proceed to tamu200_buildout_part2.ipynb
* note run_dir - we'll need it in Part2


[Apply exclusions](#apply_exclusions)

* the end
<a id='the_end'></a>

In [ ]:
run_dir

In [ ]:
print("blet")